<h2>MODELOS Y SIMULACION</h2>

TP Titanic



In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors

In [ ]:
# Opening CSV
data = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/Titanic-TP/train.csv')
# /content/drive/MyDrive/Colab Notebooks/Titanic-TP
data.head()

In [ ]:
# Cleaning
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)
# Dummies for Sex
data = pd.get_dummies(data, columns=['Sex'], drop_first=True)
# Result
data.head()

In [ ]:
explicativeVariables = data.drop('Survived', axis=1)
explicativeVariables.head()

In [ ]:
targetVariable = data[['Survived']]
targetVariable.head()

In [ ]:
# Function for graphing
def graph(x, y, survived, xLabel, yLabel):
    plt.scatter(x, y, c=np.array(survived))
    plt.xlabel(xLabel)
    plt.ylabel(yLabel)
    plt.colorbar()
    plt.show()

In [ ]:
def graphRelationBetweenTwoVariables(explicativeVariables, targetVariable):
    for i in range(0, explicativeVariables.columns.size - 1):
        for j in range(i + 1, explicativeVariables.columns.size):
            columnI = explicativeVariables.columns[i]
            columnJ = explicativeVariables.columns[j]
            graph(explicativeVariables[columnI], explicativeVariables[columnJ], targetVariable, columnI, columnJ)

In [ ]:
graphRelationBetweenTwoVariables(explicativeVariables, targetVariable)